# *checkpoint_schedule* application: Adjoint-Based Gradient with Burger's Equation

This user example shows adjoint-based gradient computation using the *checkpointing_schedules* package. We initially define the adjoint-based gradient problem and then present the forward and adjoint solvers prescribed by the *checkpointing_schedules* package.

## Defining the application

Let us consider a one-dimensional (1D) problem aiming to compute the gradient/sensitivity of an objective functional $I$ with respect to a control parameter. The objective functional is given by the expression:

$$
I(u_0) = \int_{\Omega} \frac{1}{2} u(x, \tau)u(x, \tau) \, dx
\tag{1}
$$

This measures the energy of a 1D velocity variable $u = u(x, \tau)$ at a time $\tau$, where $u$ is governed by the 1D viscous Burgers equation, a non-linear equation for the advection and diffusion of momentum:

$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} = 0.
\tag{2}
$$

Here, $x \in [0, L]$ is the space variable, and $t \in \mathbb{R}^{+}$ represents the time variable. The boundary condition is $u(0, t) = u(L, t) = 0$, where $L$ is the length of the 1D domain. The initial condition is given by $u_0 = \sin(\pi x)$.

The control parameter is the initial condition $u_0$. Hence, the objective is to compute the adjoint-based gradient of the functional $I(u_0)$ with respect to $u_0$.

This example sets the adjoint equation from the continuous formulation, meaning the adjoint PDE is obtained from the continuous forward PDE (Partial Differential Equation). The adjoint-based gradient is given by the expression:

$$
\nabla_{u_0} I = \int_{\Omega}  \lambda(x, 0) \delta u_0 \, dx,
\tag{3}
$$

where $\lambda(x, 0)$ is the adjoint variable governed by the adjoint system:

$$
\frac{\partial \lambda}{\partial t} - \lambda \frac{\partial u}{\partial x} + u \frac{\partial \lambda}{\partial x} + \nu \frac{\partial^2 \lambda}{\partial x^2} = 0,
\tag{4}
$$

satisfying the boundary condition $\lambda (0, t) = \lambda(L, t) = 0$. In this case, the initial condition is $\lambda (x, \tau) = u(x, \tau)$.

## Burger's equation solver

The sensitivity computation is performed by solving the forward and adjoint equations. The forward equation is 1D viscous Burgers equation (2). For this current sensitivity problem, the adjoint equation is given by (4). 

To solve forward and adjoint solvers, we implement `BurgersEquation` class that execute the forward and adjoint solvers. In addition, the `BurgersEquation` has the `copy_data` that copies the data from one storage type to another, and `adjoint_initial_condition` that sets the adjoint initial condition.

Both the forward and adjoint systems are discretised using the Finite Element Method (FEM). We use the first-order Lagrange basis functions to discretise the spatial domain. The backward finite difference method is employed to discretise the equations in time.

After discretising, the forward and adjoint systems are written in the following form:
$$M \frac{\mathbf{U}^{n+1} - \mathbf{U}^n}{\Delta t} - K \mathbf{U}^{n+1} + C(\mathbf{U}^{n+1}) \mathbf{U}^{n+1} = 0,$$
$$M \frac{\boldsymbol{\Lambda}^{n} - \boldsymbol{\Lambda}^{n+1}}{\Delta t} - K \boldsymbol{\Lambda}^{n} + C(\mathbf{U}^{n})^T \boldsymbol{\Lambda}^{n} + C_1(\mathbf{U}^{n}) \boldsymbol{\Lambda}^{n} = 0,$$
where $M$, $K$ are the mass and stiffness. $U$ represents the solution vector, and $\Lambda$ is the adjoint solution vector. The matrices $C$ and $C_1$ are obtained from the convection terms. The superscript $n$ denotes the time step, and $\Delta t$ is the time step size. The solution vector $\mathbf{U}^{n+1}$ is obtained by solving the non-linear forward system using the Newton method. 

In [1]:
import numpy as np
from enum import Enum
import os
from scipy.sparse.linalg import spsolve
from scipy.sparse import lil_matrix
from scipy.optimize import newton


class BurgersEquation:
    """This class is capable to solve the time-dependent forward 
    and adjoint burger's equation.

    Parameters
    ----------
    model : dict
        The model parameters containing the essential information to solve
        the burger's equation.
    init_condition : array
        The initial condition used to solve the forward burger's equation.
    mesh : array
        The spatial mesh.
    """
    def __init__(self, model, forward_initial_condition, mesh):
        self.model = model
        self.mesh = mesh
        self.forward_work_memory = {StorageType.WORK: {}}
        self.forward_work_memory[StorageType.WORK][0] = forward_initial_condition
        self.forward_final_solution = None
        self.adjoint_work_memory = {StorageType.WORK: {}}
        self.restart_forward = {StorageType.RAM: {}, StorageType.DISK: {}}
        self.adjoint_dependency = {StorageType.WORK: {}, StorageType.RAM: {}, StorageType.DISK: {}}
        self.mode = "forward"

    def forward(
            self, n0, n1, storage=None, write_adj_deps=False,
            write_ics=False, single_storage=False
    ):
        """Solve the non-linear forward burger's equation in time.

        Parameters
        ----------
        n0 : int
            Initial time step.
        n1 : int
            Final time step.
        storage : StorageType, optional
            The storage type, which can be StorageType.RAM, StorageType.DISK,
            StorageType.WORK, or StorageType.NONE.
        write_adj_deps : bool, optional
            Whether the adjoint dependency data will be stored.
        write_ics : bool, optional
            Whether the forward restart data will be stored.
        single_storage : bool, optional
            This parameter is used to indicated whether a checkpointing schedule
            is single storage or not. Single storage means that no checkpointing
            algorithm (eg, `Revolve`, `HRevole`) is employed. 
        """
        M = self._mass_matrix()
        K = self._stiffness_matrix()

        def _non_linear(u_new, u):
            """Define the non-linear system.

            Parameters
            ----------
            u_new : array
                Forward solution at the `n + 1` time step.
            u : array
                Forward solution at the `n` time step.
            """
            C = self._convection_matrix(u_new)
            # Set the boundary conditions.
            u[0] = u[self.model["nx"] - 1] = 0
            F = (M + self.model["dt"] * (- K + C)) * u_new - M * u
            return F

        # Get the initial condition
        u = self.forward_work_memory[StorageType.WORK][n0]
        del self.forward_work_memory[StorageType.WORK][n0]
        if write_ics:
            self.restart_forward[storage][n0] = u.copy()
        u_new = u.copy()
        n1 = min(n1, self.model["max_n"])
        step = n0
        if write_adj_deps:
            self.adjoint_dependency[storage][step] = u
        while step < n1:    
            u_new = newton(lambda u_new: _non_linear(u_new, u), u)
            step += 1
            u = u_new.copy()
            if write_adj_deps:
                self.adjoint_dependency[storage][step] = u
        if step == self.model["max_n"]:
            self.forward_final_solution = u_new.copy()
        if (not write_adj_deps 
           or (self.mode == "forward" and step < self.model["max_n"])
        ):
            self.forward_work_memory[StorageType.WORK][step] = u_new.copy()

    def adjoint(self, n0, n1, clear_adj_deps, single_storage=False):
        """Execute the adjoint equation in time.

        Parameters
        ---------
        n0 : int
            Initial time step.
        n1 : int
            Final time step.
        clear_adj_deps : bool
            If `True`, the adjoint dependency data will be cleared.
        """
        u = self.adjoint_work_memory[StorageType.WORK][n1]
        del self.adjoint_work_memory[StorageType.WORK][n1]
        u_new = np.zeros(self.model["nx"])
        steps = n1 - n0
        t = n1
        M = self._mass_matrix()
        K = self._stiffness_matrix()
        for _ in range(steps):
            u[0] = u[self.model["nx"] - 1] = 0
            C = self._convection_matrix(self.adjoint_dependency[StorageType.WORK][t - 1])
            C1 = self._convection_matrix_2(self.adjoint_dependency[StorageType.WORK][t - 1])
            A = M + self.model["dt"] * (- K + C.T + C1)
            d = M * u
            u_new = spsolve(A, d)
            u = u_new.copy()
            if clear_adj_deps:
                del self.adjoint_dependency[StorageType.WORK][t - 1]
            t -= 1
        self.adjoint_work_memory[StorageType.WORK][n0] = u_new

    def copy_data(self, step, from_storage, to_storage, move=False, single_storage=False):
        """Copy data from one storage to another.

        Parameters
        ----------
        step : int
            The time step.
        from_storage : StorageType
            The storage type from which the data will be copied.
        to_storage : StorageType
            The storage type to which the data will be copied.
        move : bool, optional
            Whether the data will be moved or not. If `True`, the data will be
            removed from the `from_storage`.
        """
        if to_storage == StorageType.WORK:
            if single_storage:
                self.adjoint_dependency[StorageType.WORK][step] = self.adjoint_dependency[from_storage][step]
            else:
                self.forward_work_memory[StorageType.WORK][step] = self.restart_forward[from_storage][step]
        else:
            self.restart_forward[to_storage][step] = self.restart_forward[from_storage][step]
        if move:
            if single_storage:
                del self.adjoint_dependency[from_storage][step]
            else:
                del self.restart_forward[from_storage][step]

    def adjoint_initial_condition(self):
        """Set the adjoint initial condition.
        """
        self.mode = "adjoint"
        u = self.forward_final_solution
        self.adjoint_work_memory[StorageType.WORK][self.model["max_n"]] = u

    def _mass_matrix(self):
        """This function assembles the mass matrix.

        Returns
        -------
        M : scipy.sparse.lil_matrix
            The mass matrix.
        
        Notes
        -----
        The mass matrix is assembled a linear spatial basis functions.
        """
        num_nodes = self.model["nx"]
        M = lil_matrix((num_nodes, num_nodes))
        M[0, 0] = M[num_nodes - 1, num_nodes - 1] = 1/3
        M[0, 1] = M[num_nodes - 1, num_nodes - 2] = 1/6
        for i in range(1, num_nodes - 1):
            M[i, i - 1] = M[i, i + 1] = 1/6
            M[i, i] = 2/3
        return M
    
    def _stiffness_matrix(self):
        """This function assembles the stiffness matrix.

        Returns
        -------
        K : scipy.sparse.lil_matrix
            The stiffness matrix.
        """
        num_nodes = self.model["nx"]
        h = self.model["lx"] / self.model["nx"] 
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        b = self.model["nu"] / (h ** 2)
        # local_stiffness = np.array([[-1, 1], [1, -1]])
        K = lil_matrix((num_nodes, num_nodes))
        K[0, 0] = K[num_nodes - 1, num_nodes - 1] = - b
        K[0, 1] = K[num_nodes - 1, num_nodes - 2] = + b
        for i in range(1, num_nodes - 1):
            K[i, i - 1] = K[i, i + 1] = b
            K[i, i] = - 2 * b
        return K
    
    def _convection_matrix(self, u, adjoint=False):
        """This function assembles the convection matrix.

        Parameters
        ----------
        u : array
            State vector.

        Returns
        -------
        C : scipy.sparse.lil_matrix
            The convection matrix.
        """
        num_nodes = self.model["nx"]
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        h = self.model["lx"] / self.model["nx"] 
        C = lil_matrix((num_nodes, num_nodes))
        C[0, 0] = - (1/3 * u[0] / h + 1/6 * u[1] / h)
        C[num_nodes - 1, num_nodes - 1] = 1/3 * u[num_nodes - 1] / h + 1/6 * u[num_nodes - 2] / h
        C[0, 1] = - C[0, 0]
        C[num_nodes - 1, num_nodes - 2] = - C[num_nodes - 1, num_nodes - 1]
        for i in range(1, num_nodes - 1):
            C[i, i - 1] = - (1/6 * u[i - 1] / h + 1/3 * u[i] / h)
            C[i, i] = (1/6 * u[i - 1] / h - 1/6 * u[i + 1] / h)
            C[i, i + 1] = (1/6 * u[i + 1] / h + 1/3 * u[i] / h)
        return C
    
    def _convection_matrix_2(self, u_forward):
        """This function assembles the convection matrix.

        Parameters
        ----------
        u_forward : _type_
            _description_
        """
        num_nodes = self.model["nx"]
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        h = self.model["lx"] / self.model["nx"] 
        C = lil_matrix((num_nodes, num_nodes))
        C[0, 0] = (- u_forward[0] + u_forward[1]) / (3 * h)
        C[0, 1] = (- u_forward[0] + u_forward[1]) / (6 * h)
        C[num_nodes - 1, num_nodes - 1] = (u_forward[num_nodes - 1] - u_forward[num_nodes - 2]) / (3 * h)
        C[num_nodes - 1, num_nodes - 2] = (u_forward[num_nodes - 1] - u_forward[num_nodes - 2]) / (6 * h)
        for i in range(1, num_nodes - 1):
            C[i, i - 1] = (- u_forward[i - 1] + u_forward[i]) / (6 * h)
            C[i, i] = (- u_forward[i - 1] + u_forward[i + 1]) / (3 * h)
            C[i, i + 1] = (u_forward[i + 1] - u_forward[i]) / (6 * h)
        return C

## Checkpointing
The adjoint PDE is then solved in a reverse time order and it depends on the forward solution (see the adjoint equation (4)). Therefore, to store the forward state variable is required. Storing the entire forward state in preparation for the adjoint calculation has a memory footprint linear in the number of time steps. For sufficiently large problems this can exhaust the
memory of a computer system. To overcome this kind of problem, checkpointing algorithms are used to reduce the memory usage.

To employ a checkpointing method in the current adjoint-based sensitivity, we define a `CheckpointingManager` to manage the execution of forward and adjoint models. The `CheckpointingManager` contains the `execute` method, which performs each action specified in the checkpoint schedule (`_schedule`). Within the `execute` method, we have the single-dispatch generic `action` function that is overloaded by particular functions. For instance, if the `checkpoint_schedule` action is `Forward`, the `action_forward` function is called. Within this function, the necessary code is implemented to progress the forward equation. In this particular example, the forward solver is executed by calling `self.equation.forward`. Here, `self.equation` is an attribute of `CheckpointingManager`. Similarly, the adjoint solver is executed by calling `self.equation.adjoint` within the `action_reverse` function.

In [2]:
import functools, sys
from checkpoint_schedules import *
class CheckpointingManager:
    """Manage the forward and adjoint solvers.

    Attributes
    ----------
    schedule : CheckpointSchedule
        The schedule created by `checkpoint_schedules` package.
    equation : object
        An equation object used to solve the forward and adjoint solvers.
    
    Notes
    -----
    The `equation` object contains methods to execute the forward and adjoint. In 
    addition, it contains methods to copy data from one storage to another, and
    to set the initial condition for the adjoint.
    """
    def __init__(self, schedule, equation):
        self.max_n = sys.maxsize
        self.equation = equation
        self.reverse_step = 0
        self._schedule = schedule
        self.single_storage = False
        if (
                isinstance(self._schedule, SingleMemoryStorageSchedule) 
                or isinstance(self._schedule, SingleDiskStorageSchedule)
        ):
            self.single_storage = True
        
    def execute(self):
        """Execute forward and adjoint using checkpointing.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            n1 = cp_action.n1
            self.equation.forward(cp_action.n0, n1, storage=cp_action.storage,
                                  single_storage=self.single_storage,
                                  write_adj_deps=cp_action.write_adj_deps,
                                  write_ics=cp_action.write_ics)
            if n1 >= self.equation.model["max_n"]:
                n1 = min(n1, self.equation.model["max_n"])
                self._schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            if self.reverse_step == 0:
                self.equation.adjoint_initial_condition()
            self.equation.adjoint(cp_action.n0, cp_action.n1, cp_action.clear_adj_deps, single_storage=self.single_storage)
            self.reverse_step += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            self.equation.copy_data(cp_action.n, cp_action.from_storage, cp_action.to_storage,
                                    move=False, single_storage=self.single_storage)

        @action.register(Move)
        def action_move(cp_action):
            self.equation.copy_data(cp_action.n, cp_action.from_storage, cp_action.to_storage,
                                    move=True, single_storage=self.single_storage)
            
        @action.register(EndForward)
        def action_end_forward(cp_action):
            if self._schedule.max_n is None:
                self._schedule._max_n = self.max_n
            assert self.reverse_step == 0
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            if self._schedule.max_n != self.reverse_step:
                raise ValueError("The number of steps in the reverse phase"
                                 "is different from the number of steps in the"
                                 "forward phase.")
            
        self.reverse_step = 0
        for _, cp_action in enumerate(self._schedule):
            action(cp_action)
            if isinstance(cp_action, EndReverse):
                break

## Adjoint-based sensitivity computations

The purpose of this adjoint-based sensitivity computation is to use every checkpointing approach available in the `checkpoint_schedules` package and verify the consistent of the results. We employ a fundamental tool used in verification of gradients, which is the
*second order Taylor remainder convergence test*. Thus, let us consider the functional $I(u_0)$  and $\nabla_{u_0}$ be its gradient with respect to the control parameter $u_0$. Let $u$ be the solution of the forward problem, and let $\delta u$ be a perturbation to $u$. Therefore, the *Taylor remainder convergence test* is given by the expression: 
$$ \left|I(u + \delta u) - I(u) - \nabla_{u_0} I \cdot \delta u\right| \rightarrow 0 \quad \mathrm{at} \ O(\delta u^2).$$

In the next sections, we present the results of the adjoint-based sensitivity computation using the `checkpoint_schedules` package and the *Taylor remainder convergence test*. It is expected the convergence rate of the *Taylor remainder convergence test* is approximately $2$ in every checkpointing approach.

In [3]:
def taylor_remainder_test(adjoint_solution, J, u0):
    epsilons = [0.05 / 2**i for i in range(4)]
    E = []
    h = u0
    for eps in epsilons:
        up = u0 + h * eps
        dJdm = sum(adjoint_solution[StorageType.WORK][0].reshape(-1) * h.reshape(-1)) * 1 / model["nx"]
        burgers = BurgersEquation(model, up, mesh)
        burgers.forward(0, model["max_n"])
        # Functional value at the perturbed state.
        u2 = burgers.forward_final_solution**2
        Jp = sum(u2.reshape(-1) * 0.5) * 1 / model["nx"]
        E.append(abs(Jp - J - dJdm * eps))
    return E, epsilons

def convergence_rates(E_values, eps_values, show=True):
    from numpy import log
    r = []
    for i in range(1, len(eps_values)):
        r.append(log(E_values[i] / E_values[i - 1])
                 / log(eps_values[i] / eps_values[i - 1]))
    if show:
        print("Computed convergence rates: {}".format(r))
    return r

Below, we define the `model` dictionary containing the parameters required for the forward and adjoint solvers. The `model` dictionary is then passed to the `BurgersEquation` class. Additionally, we set up the 1D mesh and the initial condition for the forward Burgers' solver.

In [4]:
model = {"lx": 1,   # lenght domain
         "nx": 100,  # number of nodes
         "dt": 0.0005,  # time step
         "nu": 0.01,  # viscosity
         "max_n": 500,  # total steps
         "chk_ram": 50,  # number of checkpoints in RAM
         "chk_disk": 50,  # number of checkpoints on disk
        }
mesh = np.linspace(0, model["lx"], model["nx"]) # create the spatial grid
u0 = np.sin(np.pi*mesh) # initial condition

The adjoint-based sensitivity is initially computed using the `SingleMemoryStorageSchedule` checkpointing approach. The `SingleMemoryStorageSchedule` stores the forward data of each time-step in working memory. As explained in the [notebook with illustrative example](https://nbviewer.org/github/firedrakeproject/checkpoint_schedules/blob/main/docs/notebooks/tutorial.ipynb), this schedule does not require the maximal step (`model["max_n"]`).

In [5]:
schedule = SingleMemoryStorageSchedule()  # Create the checkpointing schedule.
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
manager = CheckpointingManager(schedule, burger)  # Create the checkpointing manager.
manager.execute()  # execute the checkpointing schedule using `SingleMemoryStorageSchedule` schedule.
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

/Users/ddolci/work/my_venv/lib/python3.11/site-packages/scipy/optimize/_zeros_py.py:482: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)


Computed convergence rates: [1.99360272596985, 1.9872798748302887, 1.9378150121167665]
Adjoint-based sensitivity: 0.45823268843319326


The following example shows the usage of the `SingleDiskStorageSchedule` schedule. In this case, the forward data used in the adjoint computations is only stored on disk. The `SingleDiskStorageSchedule` schedule does not require the definition of the maximal step `model["max_n"]` before the execution of the forward solver. 

In [6]:
schedule = SingleDiskStorageSchedule()  # Create the checkpointing schedule.
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
manager = CheckpointingManager(schedule, burger)  # Create the checkpointing manager.
manager.execute()  # execute the checkpointing schedule using `SingleMemoryStorageSchedule` schedule.
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


With the example above, we do not move any data from the disk to the work in memory, i.e., we copy the data from the disk and keep it on disk. The next example shows the usage of the `SingleDiskStorageSchedule` schedule with the `mode_data=True` argument. In this case, the forward data used in the adjoint compuations stored on disk is moved to the work in memory, i.e., we copy the data from the disk and remove it from the disk. 

In [7]:
schedule = SingleDiskStorageSchedule(move_data=True)  # Create the checkpointing schedule.
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
manager = CheckpointingManager(schedule, burger)  # Create the checkpointing manager.
manager.execute()  # execute the checkpointing schedule using `SingleMemoryStorageSchedule` schedule.
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


The following example uses the `Revolve` schedule [1]. The `Revolve` algorithm requires the definition of the maximal step `model["max_n"]` before the execution of the forward solver, and also the specification of the number of checkpoints stored in RAM (Random Access Memory).

In [8]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = Revolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


The `DiskRevolve` [3] schedule requires the definition of the maximal step `model["max_n"]` before the execution of the forward solver, and the maximum number of checkpoints to be saved in RAM. This schedule automatically computes the number of checkpoints to store on disk, considering factors such as:
- Computational cost to execute the forward solver in one step.
- Computational cost to store the forward data on disk.

In [9]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = DiskRevolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


The `PeriodicDiskRevolve` [4] schedule also requires the definition of the maximal step `model["max_n"]` before the execution of the forward solver. This schedule automatically calculates the number of steps stored on disk.

In [10]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = PeriodicDiskRevolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

We use periods of size  51
Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


In the following examples, we employ the `HRevolve` [5] and `MultistageCheckpointSchedule` [2] schedules. These checkpointing schedules require the definition of the maximal step `model["max_n"]` before the execution of the forward solver. In these schedules, we specify the maximum number of checkpoints to be stored on disk (`model["chk_disk"]`).

In [11]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = HRevolve(model["max_n"], model["chk_ram"], model["chk_disk"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


In [12]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = MultistageCheckpointSchedule(model["max_n"], model["chk_ram"], model["chk_disk"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


`TwoLevelCheckpointSchedule` [6] does not require the maximal step `model["max_n"]` to be defined before the execution of the forward solver, saves the forward restart data on disk based on the `period` argument. For instance, if `period = 10`, the forward restart data is stored every ten steps on disk.

During the adjoint computation, the user can define additional storage of the forward restart data. This is carried out according to the `binomial_storage` argument. The additional number of checkpoints stored is set by the second argument of the `TwoLevelCheckpointSchedule` class. In the example below, we set `model["chk_ram"]` as the additional number of checkpoints to be stored during the adjoint computation, and the storage type is in RAM (`binomial_storage=StorageType.RAM`).

In [13]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
period = 10
schedule = TwoLevelCheckpointSchedule(period, model["chk_ram"], binomial_storage=StorageType.RAM) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


Now, we define to store the additional forward restart data on disk (`binomial_storage=StorageType.DISK`).

In [14]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
period = 10
schedule = TwoLevelCheckpointSchedule(period, model["chk_disk"], binomial_storage=StorageType.DISK) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
u_2 = burger.forward_final_solution**2  # Compute the energy.
J = sum(u_2.reshape(-1) * 0.5) * 1/burger.model["nx"]  # Compute the functional value.
E, h = taylor_remainder_test(burger.adjoint_work_memory, J, u0)  # Compute the Taylor remainder.
convergence_rates(E, h, show=True)
senstivity = sum(burger.adjoint_work_memory[StorageType.WORK][0].reshape(-1) * u0.reshape(-1)) * 1 / model["nx"]
print("Adjoint-based sensitivity: {}".format(senstivity))

Computed convergence rates: [1.9935945213147765, 1.9872476910156494, 1.9376937399361267]
Adjoint-based sensitivity: 0.45823268734941364


## Final Remarks
Every adjoint-based sensitivity with different schedules presented consistent results. The convergence rate of the *Taylor remainder convergence test* is approximately $2$ for every case, and the adjoint-based sensitivity has the same value.

### References

[1] Griewank, A., & Walther, A. (2000). Algorithm 799: revolve: an implementation of checkpointing for the reverse or adjoint mode of computational differentiation. ACM Transactions on Mathematical Software (TOMS), 26(1), 19-45., doi: https://doi.org/10.1145/347837.347846

[2] Stumm, P., & Walther, A. (2009). Multistage approaches for optimal offline checkpointing. SIAM Journal on Scientific Computing, 31(3), 1946-1967. https://doi.org/10.1137/080718036

[3] Aupy, G., Herrmann, J., Hovland, P., & Robert, Y. (2016). Optimal multistage algorithm for adjoint computation. SIAM Journal on Scientific Computing, 38(3), C232-C255. DOI: https://doi.org/10.1145/347837.347846.

[4] Aupy, G., & Herrmann, J. (2017). Periodicity in optimal hierarchical checkpointing schemes for adjoint computations. Optimization Methods and Software, 32(3), 594-624. doi: https://doi.org/10.1080/10556788.2016.1230612

[5] Herrmann, J. and Pallez (Aupy), G. (2020). H-Revolve: a framework for adjoint computation on synchronous hierarchical platforms. ACM Transactions on Mathematical Software (TOMS), 46(2), 1-25. DOI: https://doi.org/10.1145/3378672.

[6] Pringle, G. C., Jones, D. C., Goswami, S., Narayanan, S. H. K., and  Goldberg, D. (2016). Providing the ARCHER community with adjoint modelling tools for high-performance oceanographic and cryospheric computation. https://nora.nerc.ac.uk/id/eprint/516314.